<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_3_part_3/blob/main/%D0%94%D0%97_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
import joblib

In [42]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

In [43]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [44]:
df = df[['pclass', 'age', 'embarked', 'fare']]
df.head()

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
1,1,38.0,C,71.2833
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500


# Завдання 1
Очистіть дані від викидів(лише `fare`), розділіть на тренувальну та тестову чатини

In [45]:
df.describe()

,pclass,age,fare
count,891.000000,714.000000,891.000000
mean,2.308642,29.699118,32.204208
std,0.836071,14.526497,49.693429
min,1.000000,0.420000,0.000000
25%,2.000000,20.125000,7.910400
50%,3.000000,28.000000,14.454200
75%,3.000000,38.000000,31.000000
max,3.000000,80.000000,512.329200


In [72]:
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['fare'] >= (Q1 - 1.5 * IQR)) & (df['fare'] <= (Q3 + 1.5 * IQR))]

In [73]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.head()
test_df.head()

,pclass,age,embarked,fare
604,1,35.0,C,26.5500
427,2,19.0,S,26.0000
152,3,55.5,S,8.0500
288,2,42.0,S,13.0000
358,3,NaN,Q,7.8792


# Завдання 2
Створіть Pipeline для обробки даних

In [74]:
numeric_features = ['age']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [75]:
categorical_features = ['pclass', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [76]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print(preprocessor)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pclass', 'embarked'])])


# Завдання 3
Створіть остаточну модель та натренуйте її

In [77]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [78]:
X_train = train_df.drop('fare', axis=1)
y_train = train_df['fare']
X_test = test_df.drop('fare', axis=1)
y_test = test_df['fare']

In [79]:
print("Стовпці у тренувальній вибірці:", X_train.columns)
print("Стовпці у тестовій вибірці:", X_test.columns)

Стовпці у тренувальній вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')
Стовпці у тестовій вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')


In [80]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pclass', 'embarked'])])),
                ('regressor', LinearRegression())])

# Завдання 4
Виведіть основні метрики моделі

In [82]:
y_pred = model.predict(X_test)

In [83]:
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R^2 Score: {r2_score(y_test, y_pred)}")

Mean Squared Error: 90.9589542261205
R^2 Score: 0.36848976500034625


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

Попробуйте:
* не видаляти викиди
* змінити обробку даних(imputer та кодування категоріальних даних)
* змінити параметри дерева(глибина, кількість точок у листках тощо

In [85]:
tree_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', DecisionTreeRegressor(max_depth=5))])

In [86]:
tree_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pclass', 'embarked'])])),
                ('regressor', DecisionTreeRegressor(max_depth=5))])

In [87]:
y_pred_tree = tree_model.predict(X_test)

In [88]:
print(f"Decision Tree Mean Squared Error: {mean_squared_error(y_test, y_pred_tree)}")
print(f"Decision Tree R^2 Score: {r2_score(y_test, y_pred_tree)}")

Decision Tree Mean Squared Error: 69.47148332396782
Decision Tree R^2 Score: 0.5176730742678788


# Завдання 6
Збережіть модель

In [90]:
joblib.dump(tree_model, 'ship_passenger_fare_model.pkl')

['ship_passenger_fare_model.pkl']

In [91]:
loaded_model = joblib.load('ship_passenger_fare_model.pkl')

In [92]:
data = pd.DataFrame({'pclass': [3],
                     'age': [22],
                     'embarked': ['S']})

In [93]:
predicted_fare = loaded_model.predict(data)
print(f"Прогнозована ціна: {predicted_fare[0]}")

Прогнозована ціна: 9.465427027027022
